# Script for unziping compressed sub files

Script for automatically:
* renaming movies in Downloads unless in progress to be downloaded
* renaming any present .srt files to movie file name
* unzipping any .zip sub files and finding proper directory
* all subbed movies collected to separate directory

In [1]:
# imports
import os, sys, re
import zipfile as zf
import shutil

In [125]:
def unzip_rename(f_path=os.getcwd()):
    """Move subs zip files to correct folders, 
    unpack & rename to video filename."""
    
    # if such a file/folder exists
    if os.path.exists(f_path) == False:
        print(f'There is no such directory: {f_path}')
        return 0
    
    ### RENEMAE DOWNLOADED MOVIE DIRS ##
    
    # regex patterns
    name_p = re.compile(r"^(?=.*\d{4})[A-Za-z\s\.'-]+[^\(\.\s\-]\b", re.I)
    part_p = re.compile(r"\.part$")
    year_p = re.compile(r"19\d{2}|20\d{2}")
    sep_p = re.compile(r"\s|\.|\_|\-")
    for file in os.listdir(f_path): 
        
        if name_p.match(file) is not None and os.path.isdir(f"{f_path}/{file}"):
            # skip directories that are not fully downloaded
            try:
                for inner_file in os.listdir(f"{f_path}/{file}"):
                    # if .part file excists in directory
                    if part_p.search(inner_file) is not None:
                        raise StopIteration
            except StopIteration:
                continue # skip "file" directory
            else:
                # extracts the year & name from the file
                year = re.search(year_p, file)
                name_raw = re.match(name_p, file)
                name = re.sub(sep_p, ' ', name_raw[0].lower())
                
                if os.path.exists(f"{f_path}/{file}"):
                    os.rename(f"{f_path}/{file}", f"{f_path}/{year[0]}--{name}")
    
    ### RENAME MOVIE FILES IN DIRS ###
    
    start_p = re.compile(r'^(?!.*\[s\]$)(1|2)\d{3}--')
    mp4mkv_p = re.compile(r"\.mp4$|\.avi$|\.mkv$")
    for file in os.listdir(f_path):
        
        if start_p.match(file) is not None and os.path.isdir(f"{f_path}/{file}"):
            for inner_f in os.listdir(f"{f_path}/{file}"):
        
                if mp4mkv_p.search(inner_f) is not None:
                    os.rename(f"{f_path}/{file}/{inner_f}", 
                              f"{f_path}/{file}/{file}{inner_f[-4:]}")
    
    ### CHECK IF SUBS EXCIST IN DIR AND RENAME ###
    
    srt_p = re.compile(r"\.srt$") 
    for file in os.listdir(f_path): 
        
        if start_p.match(file) is not None:
            for inner_f in os.listdir(f"{f_path}/{file}"):
                
                # if name-matching .srt file excists
                if inner_f == f"{file}.srt":
                    # rename main folder
                    os.rename(f"{f_path}/{file}", f"{f_path}/{file} [s]")
                    break
                
                # if .srt filee.sub(sep_p, ' ', name_raw[0].lower()
                elif srt_p.search(inner_f) is not None: 
                    # rename .srt file
                    os.rename(f"{f_path}/{file}/{inner_f}", 
                              f"{f_path}/{file}/{file}.srt")
                    # rename main folder
                    os.rename(f"{f_path}/{file}", f"{f_path}/{file} [s]")
                    break
                    
    ### CHECK IF SUB FOLDER EXCISTS ###    
    
    subs_dir_p = re.compile(r"^[Ss]ubs?|^[Oo]ther", re.I)
    eng_sub_p = re.compile(r"^(?=.*\.srt$).*(eng?l?i?s?h?)", re.I)
    for file in os.listdir(f_path): 
        # find valid dirs with no subs yet
        if start_p.match(file) is not None and \
        os.path.isdir(f"{f_path}/{file}") == True:
            #print(file)
            try:
                for inner_f in os.listdir(f"{f_path}/{file}"):
                    # if folder Subs excists
                    if subs_dir_p.search(inner_f) is not None:
                        for sub in os.listdir(f"{f_path}/{file}/{inner_f}"):
                            
                            if eng_sub_p.search(sub) is not None:
                                # rename subs to movie file name
                                os.rename(f"{f_path}/{file}/{inner_f}/{sub}",
                                          f"{f_path}/{file}/{file}.srt")
                                # rename main filename
                                os.rename(f"{f_path}/{file}",
                                          f"{f_path}/{file} [s]")
                                raise StopIteration # break out of the loop
            except StopIteration:
                continue # continue iterating over main files
    
    ### COPY AVAILABLE ZIPS TO FOLDERS - UNZIP - RENAME ##
    
    # capture ("movie_full_name", "movie_words") tuple for movies not subbed
    movies_left_p = re.compile(r"^(?!.*\[s\]$)(?:[12]\d{3}--)(.+)")
    movies_left = [(movies_left_p.search(file)[0], 
                    movies_left_p.search(file)[1].split(" "))  
                    for file in os.listdir(f_path) \
                    if movies_left_p.search(file) is not None]

    # unzip .zip file to appropriate movie dir
    zip_p = re.compile(r"(?=.+\.zip$)[^\d\W_]+")
    for file in os.listdir(f_path):
        if zip_p.search(file) is not None:
            # all word parts of the .zip file
            zip_words = [word.lower() for word in zip_p.findall(file)]
            try:
                # find its movie dir
                for movie in movies_left:
                    if all(word in zip_words for word in movie[1]):
                        # extract all to movie[0] dir
                        with zf.ZipFile(f"{f_path}/{file}", 'r') as zip_ref:
                            # find the first .srt file
                            for fileName in zip_ref.namelist():
                                if fileName.endswith('.srt'):
                                    # extract srt to movie folder
                                    zip_ref.extract(fileName, f"{f_path}/{movie[0]}")
                                    # rename the .srt file in movie folder
                                    os.rename(f"{f_path}/{movie[0]}/{fileName}",
                                              f"{f_path}/{movie[0]}/{movie[0]}.srt")
                                    # delete the .zip file 
                                    os.remove(f"{f_path}/{file}")
                                    # rename movie dir
                                    os.rename(f"{f_path}/{movie[0]}",
                                              f"{f_path}/{movie[0]} [s]")
                                    raise StopIteration
            except StopIteration:
                continue
    
    ### UNZIP ALL REMAINING .ZIP FILES TO DOWNLOADS ###
    for file in os.listdir(f_path):
        if zip_p.search(file) is not None:
            with zf.ZipFile(f"{f_path}/{file}", 'r') as zip_ref:
                # find the first .srt file
                for fileName in zip_ref.namelist():
                    if fileName.endswith('.srt'):
                        # extract srt to downloads dir
                        zip_ref.extract(fileName, f"{f_path}")
            # delete the .zip file 
            os.remove(f"{f_path}/{file}")
    
    ### CREATE DIR FOR SUBBED FILES ###
    
    # if such a file/folder exists
    if not os.path.exists(f"{f_path}/Movies Subbed"):
        os.makedirs(f"{f_path}/Movies Subbed")
    
    subbed_p = re.compile(r".+\[s\]$")
    # move all subbed movies to that directory
    for file in os.listdir(f_path):
        if subbed_p.match(file) is not None:
            os.rename(f"{f_path}/{file}",
                      f"{f_path}/Movies Subbed/{file[:-4]}")


if __name__ == '__main__':
    unzip_rename('/home/tonu/Downloads')